In [1]:
from datetime import date
from pathlib import Path

import numpy as np
import pandas as pd

import requests

In [2]:
URL_cumu = "https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.csv"
URL_daily = "https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.csv"
RAWPATH = Path.cwd().parent / 'raw-data' 
CLEANPATH = Path.cwd().parent / 'clean-data' 

In [3]:
def export_csv(df, name, raw=0):
    if raw:
        path = Path.cwd().parent / 'raw-data' / '{}.csv'.format(name)
    else:
        path = Path.cwd().parent / 'clean-data' / '{}.csv'.format(name)
        
    df.to_csv(path)

In [4]:
def set_nat_cumu(url):
    df_cumu = pd.read_csv(url,sep=";")
    
    export_csv(df_cumu, "cumulative_raw", raw=1)
    print("cumulative_raw exported to csv")
    
    df_cumu['Date'] = pd.to_datetime(df_cumu['Date_of_report']).dt.date
    df_cumu.drop(['Municipality_code','Municipality_name','Date_of_report'], inplace=True, axis=1)
    
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths'
    }
    
    df_cumu.rename(mapping, inplace=True, axis=1)
    
    df_cumu = df_cumu.groupby(['Date'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df_cumu,"national_cumulative_clean")
    
    print("national_cumulative_clean exported to csv")

In [5]:
def set_province_cumu(path):
    df_province = pd.read_csv(path /'cumulative_raw.csv', index_col=0)
    df_province['date'] = pd.to_datetime(df_province['Date_of_report']).dt.date
    df_province.drop(['Municipality_code','Municipality_name','Date_of_report'], inplace=True, axis=1)
    
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths',
        'Province':'province'
    }
    
    df_province.rename(mapping, inplace=True, axis=1)
    
    df_province = df_province.groupby(['date','province'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df_province,"province_cumulative_clean")
    
    print("province_cumulative_clean exported to csv")

In [6]:
def set_nat_daily(url):
    df = pd.read_csv(url,sep=";")
    
    export_csv(df, "daily_raw", raw=1)
    print("daily_raw exported to csv")
    
    df['date'] = pd.to_datetime(df['Date_of_publication']).dt.date
    df.drop(['Municipality_code','Municipality_name','Date_of_publication','Date_of_report','Province','Security_region_code','Security_region_name', 'Municipal_health_service', 'ROAZ_region'], inplace=True, axis=1)
    
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths'
    }
    
    df.rename(mapping, inplace=True, axis=1)
    
    df = df.groupby(['date'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df,"national_daily_clean")
    
    print("national_daily_clean exported to csv")
    

In [7]:
def set_province_daily(path):
    df = pd.read_csv(path /'daily_raw.csv', index_col=0)
    df['date'] = pd.to_datetime(df['Date_of_publication']).dt.date
    df.drop(['Municipality_code','Municipality_name','Date_of_publication','Date_of_report', 'Security_region_code','Security_region_name', 'Municipal_health_service', 'ROAZ_region'], inplace=True, axis=1)
        
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths',
        'Province':'province'
    }
    
    df.rename(mapping, inplace=True, axis=1)
    
    df = df.groupby(['date','province'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df,"province_daily_clean")
    
    print("province_daily_clean exported to csv")

In [8]:
set_nat_cumu(URL_cumu)
set_province_cumu(RAWPATH)

cumulative_raw exported to csv
national_cumulative_clean exported to csv
province_cumulative_clean exported to csv


In [9]:
set_nat_daily(URL_daily)
set_province_daily(RAWPATH)

daily_raw exported to csv
national_daily_clean exported to csv
province_daily_clean exported to csv
